# Mutant Statistics

In [5]:
import pandas as pd
import numpy as np
import math
import time

# EEC

In [6]:
df = pd.read_csv("../data/eec/data.csv")

df = df.dropna(subset=["Emotion"])

label = {"joy" : 1, "sadness" : 0, "fear" : 0, "anger" : 0 }

def change_label(text) :
    return label[text]

df["label"] = df["Emotion"].apply(change_label)

df = df.drop(columns=["ID", "Race", "Emotion"])
df = df.rename(columns={"Sentence":"mutant", "Template":"template", "Gender": "gender", "Emotion word": "emotion", "Person": "person"})
df

,mutant,template,person,gender,emotion,label
0,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,angry,0
1,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,furious,0
2,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,irritated,0
3,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,enraged,0
4,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,annoyed,0
...,...,...,...,...,...,...
8635,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,funny,1
8636,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,hilarious,1
8637,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,amazing,1
8638,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,wonderful,1


In [7]:
original = []
for t, e in zip(df["template"].tolist(), df["emotion"].tolist()) :
    text = t.replace("<emotion word>", e)
    text = text.replace("<emotional situation word>", e)
#     text = text.replace("himself", "<pronoun>")
#     text = text.replace("herself", "<pronoun>")
#     text = text.replace(" a ", "<det>")
#     text = text.replace(" an ", "<det>")
    original.append(text)


# for m, p in zip(df["mutant"].tolist(), df["person"].tolist()) :
#     text = m.lower().replace(p.lower() + " ", "<person> ")
#     text = m.lower().replace(p.lower(), "<person>")
#     text = text.replace("himself", "<pronoun>")
#     text = text.replace("herself", "<pronoun>")
#     text = text.replace(" a ", "<det>")
#     text = text.replace(" an ", "<det>")
#     original.append(text)

    
df["original"] = original

### Use Groupby to Group the text by Template

In [8]:
df["original"] = df["original"].astype("category")
df["template_id"] = df["original"].cat.codes

In [9]:
gb = df.groupby("template_id")

In [10]:
gb.count()

,mutant,template,person,gender,emotion,label,original
template_id,,,,,,,
0,60,60,60,60,60,60,60
1,60,60,60,60,60,60,60
2,60,60,60,60,60,60,60
3,60,60,60,60,60,60,60
4,60,60,60,60,60,60,60
...,...,...,...,...,...,...,...
135,60,60,60,60,60,60,60
136,60,60,60,60,60,60,60
137,60,60,60,60,60,60,60


In [11]:
x = gb.count()["mutant"] != 60
gb.count()[x]

,mutant,template,person,gender,emotion,label,original
template_id,,,,,,,


# BiasFinder

In [14]:
eval_dir = "imdb_using_eec_name"
df = pd.read_csv("../data/" + eval_dir + "/test.csv", header=None, sep="\t", names=["label", "mutant", "template", "original", "identifier", "type", "gender", "country"])
df

,label,mutant,template,original,identifier,type,gender,country
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",Alonzo,name,male,African American
1,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",Adam,name,male,African American
2,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",Alphonse,name,male,African American
3,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",Alan,name,male,African American
4,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",Darnell,name,male,African American
...,...,...,...,...,...,...,...,...
140837,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...",Melanie,name,female,European American
140838,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...",Tanisha,name,female,European American
140839,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...",Nancy,name,female,European American
140840,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...",Tia,name,female,European American


### Use Groupby to Group the text by Template

In [15]:
df["original"] = df["original"].astype("category")
df["template_id"] = df["original"].cat.codes

In [16]:
gb = df.groupby("template_id")

In [17]:
gb.count()

,label,mutant,template,original,identifier,type,gender,country
template_id,,,,,,,,
0,40,40,40,40,40,40,40,40
1,26,26,26,26,26,26,26,0
2,40,40,40,40,40,40,40,40
3,40,40,40,40,40,40,40,40
4,40,40,40,40,40,40,40,40
...,...,...,...,...,...,...,...,...
3725,40,40,40,40,40,40,40,40
3726,40,40,40,40,40,40,40,40
3727,40,40,40,40,40,40,40,40


In [18]:
x = gb.count()["mutant"] != 60
gb.count()[x]

,label,mutant,template,original,identifier,type,gender,country
template_id,,,,,,,,
0,40,40,40,40,40,40,40,40
1,26,26,26,26,26,26,26,0
2,40,40,40,40,40,40,40,40
3,40,40,40,40,40,40,40,40
4,40,40,40,40,40,40,40,40
...,...,...,...,...,...,...,...,...
3725,40,40,40,40,40,40,40,40
3726,40,40,40,40,40,40,40,40
3727,40,40,40,40,40,40,40,40
